In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import sys
import os
from PIL import Image
import matplotlib.pyplot as plt
import random
from sklearn.cross_validation import train_test_split

In [2]:
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [3]:
image_size = [224,224]
num_classes = 10

drop_out = 0.2
learning_rate = 0.01
epochs = 10
batch_size = 64
display_step = 1#float('inf')

In [4]:
b = np.zeros([len(y_train),num_classes])
for index, y in enumerate(y_train):
    b[index,y] = 1
y_train = b
b = np.zeros([len(y_test),num_classes])
for index, y in enumerate(y_test):
    b[index,y] = 1
y_test = b

In [5]:
X_TF = tf.placeholder(tf.float32,[None,image_size[0],image_size[1],3])
Y_TF = tf.placeholder(tf.float32, [None, num_classes])

In [ ]:
# Download and use NASNet feature vector module.
#module = hub.Module("https://tfhub.dev/google/imagenet/nasnet_large/feature_vector/1")
module = hub.Module("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/1")

features = module(X_TF)
logits = tf.layers.dense(inputs=features,units=num_classes)
prediction = tf.nn.softmax(logits)

In [7]:
cost = tf.losses.softmax_cross_entropy(Y_TF,prediction)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y_TF, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [8]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    test_images = x_test[:batch_size]
    test_labels = y_test[:batch_size]
    test_images = tf.image.resize_images(test_images,[image_size[0],image_size[1]])
    test_images = sess.run(test_images)
    
    for epoch in range(epochs):
        num_steps = int(len(x_train)/batch_size)
        for step in range(num_steps):
            batch_xs = x_train[batch_size*step:batch_size*(step+1)]
            batch_ys = y_train[batch_size*step:batch_size*(step+1)]
            batch_xs = tf.image.resize_images(batch_xs,[image_size[0],image_size[1]])
            batch_xs = sess.run(batch_xs)
            
            print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
            print(len(sess.run(tf.global_variables())))
            print(sess.run(tf.global_variables())[551])
            print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
            
            sess.run(optimizer, feed_dict={X_TF:batch_xs,Y_TF:batch_ys})
            if step % display_step == 0:
                print("step:",step,"- epoch:",epoch)
                test_pred = sess.run(prediction,feed_dict={X_TF: test_images})
                print("y_hat:",np.argmax(test_pred[:5],axis=1))
                print("y    :",np.argmax(test_labels[:5],axis=1))
                print("train_cost:",sess.run(cost,feed_dict={X_TF:batch_xs,Y_TF:batch_ys}))
                print("valid_cost:",sess.run(cost,feed_dict={X_TF:test_images,Y_TF:test_labels}))
                print("Accuracy:",sess.run(accuracy, feed_dict={X_TF:test_images,Y_TF:test_labels}))
                print("________________________________________")

    print()
    print()
    print("Optimization Finished!")
    print("Test accuracy:",sess.run(accuracy, feed_dict={X_TF:test_images,Y_TF:test_labels}))

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
552
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
step: 0 - epoch: 0
y_hat: [0 0 0 0 0]
y    : [3 8 8 0 6]
train_cost: 2.3986502
valid_cost: 2.3674002
Accuracy: 0.09375
________________________________________
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
552
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
step: 1 - epoch: 0
y_hat: [0 0 0 0 0]
y    : [3 8 8 0 6]
train_cost: 2.3986502
valid_cost: 2.3674002
Accuracy: 0.09375
________________________________________
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
552
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
step: 2 - epoch: 0
y_hat: [0 0 0 0 0]
y    : [3 8 8 0 6]
train_cost: 2.3830252
valid_cost: 2.3674002
Accuracy: 0.09375
________________________________________
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
552
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
step: 3 - epoch:

KeyboardInterrupt: 